In [96]:
import crud_operations as co
import sqlparse
from itertools import islice
import re

In [13]:
def processQueries():
    db = co.MySQLDB('pl_queries')
    
    queries = {}
    res = db.readAllQueries()
    for q in res:
        queries[int(q[0])] = q[3]
    return queries

In [14]:
queries = processQueries()

In [52]:
# Try with one query
curr = queries[1]
parsed = sqlparse.parse(curr)
print(parsed[0])
print(parsed[0].tokens)

for tok in parsed[0].tokens:
    # print(tok.ttype, type(tok))
    if (type(tok) == sqlparse.sql.Identifier):
        print(tok.value)

select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle
from Employee e natural join Department d
where (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and
        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and
        (e.empName like '_a%' or e.empName like '_i%')
order by e.empName asc, d.deptName desc
[<DML 'select' at 0x10E1B7D00>, <Whitespace ' ' at 0x10E1B7980>, <IdentifierList 'e.empN...' at 0x10E1C39D0>, <Newline ' ' at 0x10E1B7360>, <Keyword 'from' at 0x10E1B7EC0>, <Whitespace ' ' at 0x10E1B76E0>, <Identifier 'Employ...' at 0x10E1C37D0>, <Whitespace ' ' at 0x10E1B5AD0>, <Keyword 'natura...' at 0x10E1B7F30>, <Whitespace ' ' at 0x10E1B5A60>, <Identifier 'Depart...' at 0x10E1C3B50>, <Newline ' ' at 0x10E1B5360>, <Where 'where ...' at 0x10E1C33D0>, <Keyword 'order ...' at 0x10E1A2210>, <Whitespace ' ' at 0x10E1A21A0>, <IdentifierList 'e.empN...' at 0x10E1B1150>]
Employee e
Department d


In [166]:
# For first ten queries, let's substitute:
def parse(queries):
    aliases = ['a', 'b', 'c']
    
    for query in queries:
        print("New Query\n")
        content = query[1]
        print(content)
        curr_alias = 0
        
        parsed = sqlparse.parse(query[1])
        
        if len(parsed) == 1:
            prev_keyword = ""
            for tok in parsed[0].tokens:
                # print(type(tok), tok.ttype, tok.value)
                if(str(tok.ttype) == "Token.Keyword"):
                    prev_keyword = tok.value.lower()
                if(type(tok) == sqlparse.sql.Identifier) and (prev_keyword == "from" or "join" in prev_keyword):
                    # Process tok.value
                    # The order of Department vs. Employee shouldn't matter
                    alias = tok.value.split(" ")
                    print(alias)
                    if(len(alias) > 1 and (alias[-1].lower != "asc" or alias[-1].lower != "desc")):
                        content = content.replace(" " + alias[-1] + " ", " " + aliases[curr_alias] + " ")
                        content = content.replace(alias[-1] + ".", aliases[curr_alias] + ".")
                        curr_alias += 1
                    
                if(type(tok) == sqlparse.sql.IdentifierList):
                    # use regex to make sure is alias one
                    if re.match(r'[a-zA-Z ]+ [a-zA-Z ]+,*', tok.value):
                        sets = tok.value.split(",")
                        for s in sets:
                            alias = s.strip().split(" ")
                            print(alias)
                            if(len(alias) > 1 and (alias[-1].lower() != "asc" and alias[-1].lower() != "desc" and "." not in alias[-1])):
                                content = content.replace(" " + alias[-1] + ",", " " + aliases[curr_alias] + ",")
                                content = content.replace(" " + alias[-1] + "\n", " " + aliases[curr_alias] + "\n")
                                content = content.replace(alias[-1] + ".", aliases[curr_alias] + ".")
                                curr_alias += 1
            print("Parsed: ", (query[0], content))
                    
        else:
            print("more than one")
        print("=============")

In [56]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [57]:
n = 10
subset = take(n, queries.items())
print(subset)

[(1, "select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nfrom Employee e natural join Department d\nwhere (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and\n        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and\n        (e.empName like '_a%' or e.empName like '_i%')\norder by e.empName asc, d.deptName desc"), (2, "SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nFROM Employee e join Department d on e.deptID = d.deptID \nWHERE ((d.deptLocation = 'Dubai') OR (d.deptName = 'Sales')) AND (e.annualSalary >= 84000 AND e.annualSalary <= 120000) AND ((e.empName LIKE '_A%' )OR (e.empName LIKE '_I%'))\nORDER BY e.empName,  d.deptName DESC\n\n"), (3, "SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nFROM Employee e NATURAL JOIN Department d\nWHERE (d.deptLocation = 'Dubai' OR d.deptName = 'Sales') AND e.annualSalary >= 84000 AND e.annualSalary <= 120000 AND (e.empName LIKE '_A%' OR e.empName LIKE '_I%')\nORD

In [144]:
parse(subset)

New Query

select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle
from Employee e natural join Department d
where (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and
        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and
        (e.empName like '_a%' or e.empName like '_i%')
order by e.empName asc, d.deptName desc
Parsed:  (1, "select a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nfrom Employee a natural join Department d\nwhere (b.deptLocation = 'Dubai' or b.deptName = 'Sales') and\n        (a.annualSalary >= 7000 and a.annualSalary <= 100000) and\n        (a.empName like '_a%' or a.empName like '_i%')\norder by a.empName asc, b.deptName desc")
New Query

SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle
FROM Employee e join Department d on e.deptID = d.deptID 
WHERE ((d.deptLocation = 'Dubai') OR (d.deptName = 'Sales')) AND (e.annualSalary >= 84000 AND e.annualSalary <= 120000) AND ((e.empName LIKE '_A%' )OR (

In [167]:
parse(queries.items())

New Query

select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle
from Employee e natural join Department d
where (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and
        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and
        (e.empName like '_a%' or e.empName like '_i%')
order by e.empName asc, d.deptName desc
['Employee', 'e']
['Department', 'd']
Parsed:  (1, "select a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nfrom Employee a natural join Department d\nwhere (b.deptLocation = 'Dubai' or b.deptName = 'Sales') and\n        (a.annualSalary >= 7000 and a.annualSalary <= 100000) and\n        (a.empName like '_a%' or a.empName like '_i%')\norder by a.empName asc, b.deptName desc")
New Query

SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle
FROM Employee e join Department d on e.deptID = d.deptID 
WHERE ((d.deptLocation = 'Dubai') OR (d.deptName = 'Sales')) AND (e.annualSalary >= 84000 AND e.annualSalary <= 12

Parsed:  (27, "SELECt Employee.empName, Employee.annualSalary, Department.deptName, Department.deptLocation, Employee.jobTitle\nFROM Employee, Department\nWHERE (Employee.deptID = Department.deptID) AND (Department.deptLocation = 'Dubai' OR Department.deptName = 'Sales') AND (Employee.annualSalary/12 <= 10000.0 && Employee.annualSalary/12 >= 7000.0) AND (Employee.empName LIKE '_a%' OR Employee.empName LIKE '_i%')")
New Query

SELECT empName, annualSalary, deptName, deptLocation, jobTitle
FROM Department, Employee
WHERE (deptName = "Sales") AND (annualSalary/12 BETWEEN 7000 AND 10000) AND (empName LIKE '_A%' OR empName LIKE '_I%') AND jobTitle LIKE "Sales%"
ORDER BY empName ASC, deptName DESC
['empName', 'ASC']
asc
['deptName', 'DESC']
desc
Parsed:  (28, 'SELECT empName, annualSalary, deptName, deptLocation, jobTitle\nFROM Department, Employee\nWHERE (deptName = "Sales") AND (annualSalary/12 BETWEEN 7000 AND 10000) AND (empName LIKE \'_A%\' OR empName LIKE \'_I%\') AND jobTitle LIKE "Sa

['Department', 'd']
['Employee', 'e']
Parsed:  (60, 'SELECT a.deptName, a.deptLocation, COUNT(b.empID) AS cnt\nFROM Department a LEFT OUTER JOIN Employee b ON a.deptID = b.deptID\nGROUP BY a.deptID\nORDER BY cnt, a.deptName DESC')
New Query

SELECT d.deptName, d.deptLocation, count(e.empID) AS "count"
FROM Department d LEFT OUTER JOIN Employee e ON (d.deptID = e.deptID)
GROUP BY d.deptID
ORDER BY count, d.deptName DESC
['Department', 'd']
['Employee', 'e']
Parsed:  (61, 'SELECT a.deptName, a.deptLocation, count(b.empID) AS "count"\nFROM Department a LEFT OUTER JOIN Employee b ON (a.deptID = b.deptID)\nGROUP BY a.deptID\nORDER BY count, a.deptName DESC')
New Query

Select D.deptName,D.deptLocation,count(E.empID) AS num_employees
from Employee E RIGHT JOIN Department D ON E.deptID = D.deptID 
group by D.deptName,D.deptLocation
order by num_employees,D.deptName DESC

['Employee', 'E']
['Department', 'D']
Parsed:  (62, 'Select b.deptName,b.deptLocation,count(a.empID) AS num_employees\nfrom

['Salary_Grade']
Parsed:  (95, 'SELECT empName, jobTitle, salary / 12, deptID, deptName, grade\nFROM (Employee NATURAL JOIN Department) JOIN Salary_Grade \n        ON (Employee.salary >= Salary_Grade.minSalary AND Employee.salary <= Salary_Grade.maxSalary)\nWHERE salary / 12 > 7000 OR jobTitle <> "IT Support";')
New Query

SELECT e.empName, e.jobTitle, e.salary/12.0 as MontlySal, e.deptID, d.deptName, s.grade
FROM Employee e, Department d, Salary_Grade s
WHERE e.deptID = d.deptID
    AND e.salary BETWEEN s.minSalary AND s.maxSalary
    AND (((e.salary/12.0) > 7000) OR (e.jobTitle <> 'IT Support'));
['Employee', 'e']
e
['Department', 'd']
d
['Salary_Grade', 's']
s
Parsed:  (96, "SELECT a.empName, a.jobTitle, a.salary/12.0 as MontlySal, a.deptID, b.deptName, c.grade\nFROM Employee a, Department b, Salary_Grade c\nWHERE a.deptID = b.deptID\n    AND a.salary BETWEEN c.minSalary AND c.maxSalary\n    AND (((a.salary/12.0) > 7000) OR (a.jobTitle <> 'IT Support'));")
New Query

SELECT * FROM E

Parsed:  (121, 'SELECT DISTINCT e.empID, e.empName, e.jobTitle, e.managerID, e.hireDate, e.salary, e.comission, e.deptID\nFROM Salary_Grade s, Department d, Employee e\nWHERE s.grade IN (12,13) AND (e.salary BETWEEN s.minSalary AND s.maxSalary) AND e.deptID IN (SELECT d.deptID FROM Department d WHERE d.deptLocation="London")')
New Query

SELECT e.*
FROM Employee e, Salary_Grade s, Department d
WHERE ((s.grade = 12 AND e.salary > s.minSalary AND e.salary < s.maxSalary) OR (s.grade = 13 AND e.salary > s.minSalary AND e.salary < s.maxSalary)) AND e.deptID = d.deptID AND deptLocation = "London"
['Employee', 'e']
e
['Salary_Grade', 's']
s
['Department', 'd']
d
Parsed:  (122, 'SELECT a.*\nFROM Employee a, Salary_Grade b, Department c\nWHERE ((b.grade = 12 AND a.salary > b.minSalary AND a.salary < b.maxSalary) OR (b.grade = 13 AND a.salary > b.minSalary AND a.salary < b.maxSalary)) AND a.deptID = c.deptID AND deptLocation = "London"')
New Query

SELECT *
FROM Employee em
WHERE em.empID IN (SE

...

['Students']
['totCredits', 'DESC']
desc
['Students.FirstName']
students.firstname
Parsed:  (204, 'SELECT Students.FirstName,\n    Students.LastName,\n    Students.NetId,\n    (SELECT SUM(Enrollments.Credits) FROM Enrollments WHERE Enrollments.NetId = Students.NetId) AS totCredits \n    FROM Students\n    HAVING totCredits > 12\n    ORDER BY totCredits DESC, Students.FirstName')
New Query

SELECT Students.FirstName, Students.LastName, sub.NetId, sub.cred
FROM Students NATURAL JOIN (SELECT NetId, SUM(Credits) as cred
FROM Enrollments 
GROUP BY NetId
HAVING SUM(Credits) > 12) as sub
ORDER BY sub.cred DESC, Students.FirstName ASC;
['Students']
['(SELECT', 'NetId,', 'SUM(Credits)', 'as', 'cred\nFROM', 'Enrollments', '\nGROUP', 'BY', 'NetId\nHAVING', 'SUM(Credits)', '>', '12)', 'as', 'sub']
Parsed:  (205, 'SELECT Students.FirstName, Students.LastName, a.NetId, a.cred\nFROM Students NATURAL JOIN (SELECT NetId, SUM(Credits) as cred\nFROM Enrollments \nGROUP BY NetId\nHAVING SUM(Credits) > 12)

['Students', 'St']
st
['Enrollments', 'En']
en
Parsed:  (236, 'SELECT DISTINCT a.FirstName, a.LastName, a.NetId, ( SELECT SUM(Credits)\n        FROM Enrollments En2\n        WHERE En2.NetId = a.NetId\n        GROUP BY En2.NetId\n      ) AS Total_credits\nFROM Students a, Enrollments b\nWHERE ( SELECT SUM(Credits)\n        FROM Enrollments En2\n        WHERE En2.NetId = a.NetId\n        GROUP BY En2.NetId\n      ) > 12\nORDER BY Total_credits DESC, a.FirstName')
New Query

SELECT Students.FirstName, Students.LastName, Enrollments.NetId, SUM(Enrollments.Credits)
FROM Students RIGHT JOIN Enrollments
ON Students.NetId = Enrollments.NetId
GROUP BY NetId
HAVING SUM(Enrollments.Credits) > 12
ORDER BY SUM(Enrollments.Credits) DESC, Students.FirstName;

more than one
New Query

SELECT FirstName, LastName, NetId, temp2.Credits
    FROM (
    SELECT temp.NetID, temp.Credits 
    FROM (SELECT NetId, SUM(Credits) as Credits
        FROM Students NATURAL JOIN Enrollments
        GROUP BY NetId) as t

desc
['Firstname', 'ASC']
asc
Parsed:  (261, 'select FirstName, LastName,Students.NetId,sum(Enrollments.Credits) as SC\nfrom Students \nright join Enrollments\non Students.NetId=Enrollments.NetId\ngroup by Students.NetId\nhaving SC>12\norder by SC DESC,Firstname ASC\n')
New Query

SELECT s.FirstName, s.LastName, s.NetId, Sum(e.Credits) as totalCredits
FROM Students s NATURAL JOIN Enrollments e
GROUP BY s.NetId
HAVING SUM(e.Credits) > 12
ORDER BY totalCredits DESC, s.FirstName ASC
['Students', 's']
['Enrollments', 'e']
['totalCredits', 'DESC']
desc
['s.FirstName', 'ASC']
asc
Parsed:  (262, 'SELECT a.FirstName, a.LastName, a.NetId, Sum(b.Credits) as totalCredits\nFROM Students a NATURAL JOIN Enrollments e\nGROUP BY a.NetId\nHAVING SUM(b.Credits) > 12\nORDER BY totalCredits DESC, a.FirstName ASC')
New Query

SELECT  FirstName, LastName, NetId, SUM(Credits) 
FROM    Students s NATURAL JOIN Enrollments e
GROUP BY LastName, FirstName, NetId
HAVING SUM(Credits) > 12
ORDER BY SUM(Credits) DESC

['Students', 'AS', 's']
['(SELECT', 'Enrollments.NetId,', 'SUM(Enrollments.Credits)', 'AS', 'TotalCredits\n', '', '', '', '', '', 'FROM', 'Enrollments\n', '', '', '', '', '', 'GROUP', 'BY', 'Enrollments.NetId)', 'AS', 'e']
Parsed:  (296, 'SELECT a.FirstName, a.LastName, a.NetId, b.TotalCredits\nFROM Students AS s\nJOIN (SELECT Enrollmenta.NetId, SUM(Enrollmenta.Credits) AS TotalCredits\n      FROM Enrollments\n      GROUP BY Enrollmenta.NetId) AS e\nON a.NetId = b.NetId\nWHERE b.TotalCredits > 12\nORDER BY b.TotalCredits desc, a.FirstName asc')
New Query

select s.FirstName, s.LastName, s.NetId, sum(e.Credits) as temp
from Enrollments e join Students s on e.netid = s.netid
group by netid
having temp > 12
order by temp DESC, s.FirstName ASC; 
['Enrollments', 'e']
['Students', 's']
Parsed:  (297, 'select b.FirstName, b.LastName, b.NetId, sum(a.Credits) as temp\nfrom Enrollments a join Students b on a.netid = b.netid\ngroup by netid\nhaving temp > 12\norder by temp DESC, b.FirstName ASC; 

IndexError: list index out of range